In [ ]:
#| default_exp experiments.darts.metrics

In [ ]:
#| export
from typing import Optional,Union,Sequence,Tuple,Callable
import sys
import logging

import numpy as np


import  darts.metrics.metrics as darts_metrics
from darts.metrics.metrics import _get_values, _get_values_or_raise
from darts.timeseries import TimeSeries
from darts.logging import raise_if_not, get_logger

In [ ]:
#| export
def _root_mean_square_percentage_error(
    *,
    ts: np.array,
    f: np.array,
) -> float:
    """
    Computes the root mean square percentage error:

    $$ \sqrt{ (1/n) \sum  ( (ts[i] - f[i])  /ts[i] ) ^ 2 },  n* = len(ts) = len(f)$$

    .Ignores nan values and division by zero.
    n is n* minus the ignored values

    ------
    Parameters
    ------

    ts : np.array with the time-series
    f :  np.array with the forecast

    -------
    Returns
    -------
    
    """

    pe = np.divide(ts - f, ts, where=ts != 0)[ts != 0]
    
    w = ~np.isnan(pe)

    n = len(pe[w])

    return np.sqrt(np.divide(
        np.sum(np.square(pe, where=w)),
        n,
        where=n > 0,
    ))

In [ ]:
#| export

logging.disable(sys.maxsize)

In [ ]:
#| export
def _mean_squared_error(
    *,
    ts: np.array,
    f: np.array,
) -> float:
    """
    Computes the mean squared error:

    $$(1/n)\sum^{n}_{i} | ts[i] - f[i]|**2, n = len(ts) = len(f)$$

    .Ignores nan values in times-series or the forecast.

    ------
    Parameters
    ------

    ts : np.array with the time-series
    f :  np.array with the forecast

    -------
    Returns
    -------
    
    """
    d = (ts - f)**2
    w = ~np.isnan(d)
    n = len(d[w])

    return np.divide(
        np.sum(d, where=w),
        n,
        where=n > 0,
    )

In [ ]:
#| export
def _relative_root_mean_square_error(
    *,
    ts: np.array,
    f: np.array,
) -> float:
    """
    Computes the root mean square error:

    $$ \sqrt((1/n)\sum^{n}_{i}(ts[i] - f[i])**2) / ((1/n) \sum^{n}_{i} ts[i] ) ,  n = len(ts) = len(f)$$

    .Ignores nan values in times-series or the forecast.

    ------
    Parameters
    ------

    ts : np.array with the time-series
    f :  np.array with the forecast

    -------
    Returns
    -------
    
    """

    mean = np.mean(ts)
    return np.divide(np.sqrt(_mean_squared_error(ts=ts, f=f)),mean, where=mean!=0)

In [ ]:
ts, f = np.array(
    [1, 2, 3],
    dtype=float,
), np.array(
    [.5, 2.5, 2],
    dtype=float,
)
assert _relative_root_mean_square_error(ts=ts, f=f) == np.sqrt(
    np.divide((1 - .5)**2 + (2 - 2.5)**2 + (3 - 2)**2, 3)) / np.mean(ts)

In [ ]:
ts, f = np.array([1, 1]), np.array([2,3])
assert _root_mean_square_percentage_error(ts=ts, f=f) == np.sqrt( np.divide( 2**2 +1, 2))

In [ ]:
ts, f = np.array([2, 1],dtype=float), np.array([3.,3],dtype=float)
assert _root_mean_square_percentage_error(ts=ts, f=f) == np.sqrt( np.divide( (1./2.)**2 +2**2, 2))

In [ ]:
ts, f = np.array([1, 2, 3]), np.array([.5, 2.5, 2])
assert _root_mean_square_percentage_error(ts=ts, f=f) == np.sqrt(np.divide(
    (1 - .5)**2 + ((2 - 2.5) / 2)**2 + ((3 - 2) / 3.)**2, 3))

In [ ]:
ts, f = np.array(
    [1, 2, 3, 0],
    dtype=float,
), np.array(
    [.5, 2.5, 2, 5],
    dtype=float,
)
assert _root_mean_square_percentage_error(ts=ts, f=f) == np.sqrt(
    np.divide((1 - .5)**2 + ((2 - 2.5) / 2)**2 + ((3 - 2) / 3.)**2, 3))

In [ ]:
#| export
def rmspe(
    actual_series: Union[TimeSeries, Sequence[TimeSeries]],
    pred_series: Union[TimeSeries, Sequence[TimeSeries]],
    intersect: bool = True,
    *,
    reduction: Callable[[np.ndarray], float] = np.mean,
    inter_reduction: Callable[[np.ndarray], Union[float,
                                                  np.ndarray]] = lambda x: x,
    n_jobs: int = 1,
    verbose: bool = False
) -> Union[float, np.ndarray]:
    """ Root Mean Square Percentage Error (RMSPE).

    For two time series :math:`ts` and :math:`f` of length :math:`n*`, it is computed as

    .. math::\sqrt{ (1/n) \sum  ( (ts[i] - f[i])  /ts[i] ) ^ 2 }.

    If any of the series is stochastic (containing several samples), the median sample value is considered.

    Parameters
    ----------
    actual_series
        The `TimeSeries` or `Sequence[TimeSeries]` of actual values.
    pred_series
        The `TimeSeries` or `Sequence[TimeSeries]` of predicted values.
    intersect
        For time series that are overlapping in time without having the same time index, setting `intersect=True`
        will consider the values only over their common time interval (intersection in time).
    reduction
        Function taking as input a `np.ndarray` and returning a scalar value. This function is used to aggregate
        the metrics of different components in case of multivariate `TimeSeries` instances.
    inter_reduction
        Function taking as input a `np.ndarray` and returning either a scalar value or a `np.ndarray`.
        This function can be used to aggregate the metrics of different series in case the metric is evaluated on a
        `Sequence[TimeSeries]`. Defaults to the identity function, which returns the pairwise metrics for each pair
        of `TimeSeries` received in input. Example: `inter_reduction=np.mean`, will return the average of the pairwise
        metrics.
    n_jobs
        The number of jobs to run in parallel. Parallel jobs are created only when a `Sequence[TimeSeries]` is
        passed as input, parallelising operations regarding different `TimeSeries`. Defaults to `1`
        (sequential). Setting the parameter to `-1` means using all the available processors.
    verbose
        Optionally, whether to print operations progress

    Returns
    -------
    float
        Root Mean Square Percentage Error (RMSPE)

"""

    y1, y2 = _get_values_or_raise(
        actual_series, pred_series, intersect, remove_nan_union=True
    )
    return _root_mean_square_percentage_error(
        ts=y1,
        f=y2
    )



In [ ]:
#| export 
def rrmse(
    actual_series: Union[TimeSeries, Sequence[TimeSeries]],
    pred_series: Union[TimeSeries, Sequence[TimeSeries]],
    intersect: bool = True,
    *,
    reduction: Callable[[np.ndarray], float] = np.mean,
    inter_reduction: Callable[[np.ndarray], Union[float,
                                                  np.ndarray]] = lambda x: x,
    n_jobs: int = 1,
    verbose: bool = False
) -> Union[float, np.ndarray]:
    """ Relative Root Mean Square Error (rRMSE).

    For two time series :math:`ts` and :math:`f` of length :math:`n*`, it is computed as

    .. math::\sqrt((1/n)\sum^{n}_{i}(ts[i] - f[i])**2) / ((1/n) \sum^{n}_{i} ts[i] ).

    If any of the series is stochastic (containing several samples), the median sample value is considered.

    Parameters
    ----------
    actual_series
        The `TimeSeries` or `Sequence[TimeSeries]` of actual values.
    pred_series
        The `TimeSeries` or `Sequence[TimeSeries]` of predicted values.
    intersect
        For time series that are overlapping in time without having the same time index, setting `intersect=True`
        will consider the values only over their common time interval (intersection in time).
    reduction
        Function taking as input a `np.ndarray` and returning a scalar value. This function is used to aggregate
        the metrics of different components in case of multivariate `TimeSeries` instances.
    inter_reduction
        Function taking as input a `np.ndarray` and returning either a scalar value or a `np.ndarray`.
        This function can be used to aggregate the metrics of different series in case the metric is evaluated on a
        `Sequence[TimeSeries]`. Defaults to the identity function, which returns the pairwise metrics for each pair
        of `TimeSeries` received in input. Example: `inter_reduction=np.mean`, will return the average of the pairwise
        metrics.
    n_jobs
        The number of jobs to run in parallel. Parallel jobs are created only when a `Sequence[TimeSeries]` is
        passed as input, parallelising operations regarding different `TimeSeries`. Defaults to `1`
        (sequential). Setting the parameter to `-1` means using all the available processors.
    verbose
        Optionally, whether to print operations progress

    Returns
    -------
    float
        Relative Root Mean Square Error (rRMSE)

"""

    y1, y2 = _get_values_or_raise(
        actual_series, pred_series, intersect, remove_nan_union=True
    )
    return  _relative_root_mean_square_error(
        ts=y1,
        f=y2
    )